<img src="data/1_1FMoK_HWvk1IBaMelUXibw.webp" style="height:300px" />

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
import glob
import re
import os
import numpy as np
import sys
Stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Implementing helper functions

In [10]:
def finding_all_unique_words_and_freq(words):
    words_unique = []
    word_freq = {}
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
        word_freq[word] = words.count(word)
    return word_freq
def finding_freq_of_word_in_doc(word,words):
    freq = words.count(word)

def remove_special_characters(text):
    regex = re.compile('[^a-zA-Z0-9\s]')
    text_returned = re.sub(regex,'',text)
    return text_returned

<>:14: SyntaxWarning: invalid escape sequence '\s'
<>:14: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-2848781559.py:14: SyntaxWarning: invalid escape sequence '\s'
  regex = re.compile('[^a-zA-Z0-9\s]')


Finding the set of unique words from all documents of the data set

In [11]:
all_words = []
dict_global = {}
file_folder = 'data/*'
idx = 1
files_with_index = {}
for file in glob.glob(file_folder):
    print(file)
    fname = file
    file = open(file , "r")
    text = file.read()
    text = remove_special_characters(text)
    text = re.sub(re.compile('\d'),'',text)
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word for word in words if len(words)>1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in Stopwords]
    dict_global.update(finding_all_unique_words_and_freq(words))
    files_with_index[idx] = os.path.basename(fname)
    idx = idx + 1

unique_words_all = set(dict_global.keys())

<>:12: SyntaxWarning: invalid escape sequence '\d'
<>:12: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-2316475813.py:12: SyntaxWarning: invalid escape sequence '\d'
  text = re.sub(re.compile('\d'),'',text)


Defining the linked list

In [12]:
class Node:
    def __init__(self ,docId, freq = None):
        self.freq = freq
        self.doc = docId
        self.nextval = None

class SlinkedList:
    def __init__(self ,head = None):
        self.head = head

Making a linkedlist for each word and storing all the nodes (containing the file name and frequency of the respective word ) in the linkedlist.

In [13]:
linked_list_data = {}
for word in unique_words_all:
    linked_list_data[word] = SlinkedList()
    linked_list_data[word].head = Node(1,Node)
word_freq_in_doc = {}
idx = 1
for file in glob.glob(file_folder):
    file = open(file, "r")
    text = file.read()
    text = remove_special_characters(text)
    text = re.sub(re.compile('\d'),'',text)
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word for word in words if len(words)>1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in Stopwords]
    word_freq_in_doc = finding_all_unique_words_and_freq(words)
    for word in word_freq_in_doc.keys():
        linked_list = linked_list_data[word].head
        while linked_list.nextval is not None:
            linked_list = linked_list.nextval
        linked_list.nextval = Node(idx ,word_freq_in_doc[word])
    idx = idx + 1

<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-2238846192.py:11: SyntaxWarning: invalid escape sequence '\d'
  text = re.sub(re.compile('\d'),'',text)


Query processing and output generation

In [19]:
query = input('Enter your query: ')
query = word_tokenize(query)

connecting_words = []
different_words = []
bitwise_op = []

# Pisahkan antara kata dan operator
for word in query:
    if word.lower() not in ["and", "or", "not"]:
        different_words.append(word.lower())
    else:
        connecting_words.append(word.lower())

print("Operators:", connecting_words)
print("Terms:", different_words)

total_files = len(files_with_index)
zeroes_and_ones_of_all_words = []

# Buat representasi biner tiap kata
for word in different_words:
    if word.lower() in unique_words_all:
        zeroes_and_ones = [0] * total_files
        linkedlist = linked_list_data[word].head
        print("Processing:", word)
        while linkedlist.nextval is not None:
            zeroes_and_ones[linkedlist.nextval.doc - 1] = 1
            linkedlist = linkedlist.nextval
        zeroes_and_ones_of_all_words.append(zeroes_and_ones)
    else:
        print(word, " not found — dilewati.")
        # lanjut saja, jangan exit
        continue

# Jika tidak ada kata yang valid
if len(zeroes_and_ones_of_all_words) == 0:
    print("Tidak ada kata valid dalam query.")
else:
    # Proses operator Boolean
    for op in connecting_words:
        if len(zeroes_and_ones_of_all_words) < 2:
            break  # tidak cukup operand

        word_list1 = zeroes_and_ones_of_all_words.pop(0)
        word_list2 = zeroes_and_ones_of_all_words.pop(0)

        if op == "and":
            bitwise_op = [w1 & w2 for (w1, w2) in zip(word_list1, word_list2)]
        elif op == "or":
            bitwise_op = [w1 | w2 for (w1, w2) in zip(word_list1, word_list2)]
        elif op == "not":
            word_list2 = [1 - w for w in word_list2]  # NOT: ubah 1→0, 0→1
            bitwise_op = [w1 & w2 for (w1, w2) in zip(word_list1, word_list2)]

        zeroes_and_ones_of_all_words.insert(0, bitwise_op)

    # Ambil hasil akhir
    final_bits = zeroes_and_ones_of_all_words[0]
    files = []

    for i, val in enumerate(final_bits):
        if val == 1:
            files.append(files_with_index[i + 1])  # disesuaikan offset-nya

    print("Matching files:", files)

Enter your query: (badminton OR baseball) AND NOT obama
Operators: ['or', 'and', 'not']
Terms: ['(', 'badminton', 'baseball', ')', 'obama']
(  not found — dilewati.
badminton  not found — dilewati.
baseball  not found — dilewati.
)  not found — dilewati.
obama  not found — dilewati.
Tidak ada kata valid dalam query.
